# Intelligent Agents: Reflex-Based Agents for the Vacuum-cleaner World

Student Name: [Add your name]

I have used the following AI tools: [list tools]

I understand that my submission needs to be my own work: [your initials]

## Learning Outcomes

* Design and build a simulation environment that models sensor inputs, actuator effects, and performance measurement.
* Apply core AI concepts by implementing the agent function for a simple and model-based reflex agents that respond to environmental percepts.
* Practice how the environment and the agent function interact.
* Analyze agent performance through controlled experiments across different environment configurations.
* Graduate Students: Develop strategies for handling uncertainty and imperfect information in autonomous agent systems.

## Instructions

Total Points: Undergrads 98 + 5 bonus / Graduate students 110

Complete this notebook. Use the provided notebook cells and insert additional code and markdown cells as needed. Submit the completely rendered notebook as a HTML file.

### AI Use

Here are some guidelines that will make it easier for you:

* __Don't:__ Rely on AI auto completion. You will waste a lot of time trying to figure out how the suggested code relates to what we do in class. Turn off AI code completion (e.g., Copilot) in your IDE.
* __Don't:__ Do not submit code/text that you do not understand or have not checked to make sure that it is complete and correct.
* __Do:__ Use AI for debugging and letting it explain code and concepts from class.

### Using Visual Studio Code

If you use VS code then you can use `Export` (click on `...` in the menu bar) to save your notebook as a HTML file. Note that you have to run all blocks before so the HTML file contains your output.

### Using Google Colab

In Colab you need to save the notebook on GoogleDrive to work with it. For this you need to mount your google dive and change to the correct directory by uncommenting the following lines and running the code block.

In [ ]:
# from google.colab import drive
# import os
#
# drive.mount('/content/drive')
# os.chdir('/content/drive/My Drive/Colab Notebooks/')

Once you are done with the assignment and have run all code blocks using `Runtime/Run all`, you can convert the file on your GoogleDrive into HTML be uncommenting the following line and running the block.

In [ ]:
# %jupyter nbconvert --to html Copy\ of\ robot_vacuum.ipynb

You may have to fix the file location or the file name to match how it looks on your GoogleDrive. You can navigate in Colab to your GoogleDrive using the little folder symbol in the navigation bar to the left.

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different reflex-based agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square in the room has been cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To start, the agent is placed on a random square.

__Actuators:__ The agent can clean the current square (action `suck`) or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [ ]:
# make sure numpy is installed
%pip install -q numpy

In [ ]:
import numpy as np

actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty):
    return np.random.choice(actions)

In [ ]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty)

np.str_('east')

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall repeatedly or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

We implement a simple simulation environment that supplies the agent with its percepts.
The simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns a different performance measure than the one specified in the PEAS description! Since the room is infinite and all squares are constantly dirty, the agent can never clean the whole room. Your implementation needs to implement the **correct performance measure.** The energy budget of the agent is specified as `max_steps`.

In [ ]:
def simple_environment(agent_function, max_steps, verbose = True):
    num_cleaned = 0

    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent_function(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action)

        if (action == "suck"):
            num_cleaned = num_cleaned + 1

    return num_cleaned



Do one simulation run with a simple randomized agent that has enough energy for 20 steps.

In [ ]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: north
step 1 - action: north
step 2 - action: south
step 3 - action: east
step 4 - action: east
step 5 - action: east
step 6 - action: suck
step 7 - action: east
step 8 - action: suck
step 9 - action: south
step 10 - action: south
step 11 - action: east
step 12 - action: suck
step 13 - action: south
step 14 - action: west
step 15 - action: suck
step 16 - action: north
step 17 - action: west
step 18 - action: west
step 19 - action: north


4

# Tasks

## General [10 Points]

1. Make sure that you use the latest version of this notebook.
2. Your implementation can use libraries like math, numpy, scipy, but not libraries that implement intelligent agents or complete search algorithms. Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design.
3. You notebook needs to be formatted professionally.
    - Add additional markdown blocks for your description, comments in the code, add tables and use mathplotlib to produce charts where appropriate
    - Do not show debugging output or include an excessive amount of output.
    - Check that your submitted file is readable and contains all figures.
4. Document your code. Use comments in the code and add a discussion of how your implementation works and your design choices.


## Task 1: Implement a simulation environment [20 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty. ([Help with random numbers and arrays in Python](https://github.com/mhahsler/CS7320-AI/blob/master/HOWTOs/random_numbers_and_arrays.ipynb))
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment should be a function like the `simple_environment()` and needs to work with the simple randomized agent program from above. **Use the same environment for all your agent implementations in the tasks below.**

*Note on debugging:* Debugging is difficult. Make sure your environment prints enough information when you use `verbose = True`. Also, implementing a function that the environment can use to displays the room with dirt and the current position of the robot at every step is very useful.  

In [ ]:
import numpy as np
import random

# Các action có thể
ACTIONS = ["north", "east", "west", "south", "suck"]

def vacuum_environment(agent_function, n=5, p=0.2, max_steps=200, verbose=True):
    """
    Môi trường vacuum cleaner mô phỏng theo PEAS.

    Parameters:
        agent_function : hàm agent (ví dụ simple_randomized_agent)
        n : kích thước phòng n x n
        p : xác suất ô bẩn ban đầu
        max_steps : số bước tối đa (năng lượng)
        verbose : nếu True sẽ in trạng thái từng bước

    Returns:
        steps_taken : số bước agent đã dùng để dọn sạch phòng
    """
    # Khởi tạo môi trường: ma trận dirty (True=dirty, False=clean)
    room = np.random.choice([True, False], size=(n,n), p=[p, 1-p])

    # Chọn vị trí ban đầu ngẫu nhiên
    agent_pos = [random.randint(0, n-1), random.randint(0, n-1)]

    steps_taken = 0

    while steps_taken < max_steps:
        r, c = agent_pos

        # Cảm biến dirt sensor
        dirty = room[r, c]

        # Cảm biến bumper
        bumpers = {
            "north": (r == 0),
            "south": (r == n-1),
            "west":  (c == 0),
            "east":  (c == n-1)
        }

        # Gọi agent để lấy action
        action = agent_function(bumpers, dirty)

        if verbose:
            print(f"Step {steps_taken}: pos=({r},{c}), dirty={dirty}, action={action}")

        # Thực hiện action
        if action == "suck":
            room[r, c] = False  # làm sạch ô hiện tại
        elif action == "north" and not bumpers["north"]:
            agent_pos[0] -= 1
        elif action == "south" and not bumpers["south"]:
            agent_pos[0] += 1
        elif action == "west" and not bumpers["west"]:
            agent_pos[1] -= 1
        elif action == "east" and not bumpers["east"]:
            agent_pos[1] += 1
        # Nếu agent chọn đi vào tường, thì agent không di chuyển

        steps_taken += 1

        # Kiểm tra nếu tất cả ô đều sạch
        if not room.any():
            if verbose:
                print(f" All squares cleaned in {steps_taken} steps.")
            break

    return steps_taken


Show that your environment works with the simple randomized agent from above.

In [ ]:
# Hàm agent mẫu (random, không thông minh)
def simple_randomized_agent(bumpers, dirty):
    return np.random.choice(ACTIONS)

# Thử chạy mô phỏng
steps_used = vacuum_environment(simple_randomized_agent, n=5, p=0.2, max_steps=100, verbose=True)
print("Total steps:", steps_used)


Step 0: pos=(4,4), dirty=False, action=south
Step 1: pos=(4,4), dirty=False, action=east
Step 2: pos=(4,4), dirty=False, action=east
Step 3: pos=(4,4), dirty=False, action=east
Step 4: pos=(4,4), dirty=False, action=west
Step 5: pos=(4,3), dirty=False, action=east
Step 6: pos=(4,4), dirty=False, action=north
Step 7: pos=(3,4), dirty=False, action=north
Step 8: pos=(2,4), dirty=False, action=west
Step 9: pos=(2,3), dirty=False, action=north
Step 10: pos=(1,3), dirty=False, action=suck
Step 11: pos=(1,3), dirty=False, action=west
Step 12: pos=(1,2), dirty=False, action=north
Step 13: pos=(0,2), dirty=True, action=suck
Step 14: pos=(0,2), dirty=False, action=suck
Step 15: pos=(0,2), dirty=False, action=north
Step 16: pos=(0,2), dirty=False, action=east
Step 17: pos=(0,3), dirty=False, action=south
Step 18: pos=(1,3), dirty=False, action=south
Step 19: pos=(2,3), dirty=False, action=north
Step 20: pos=(1,3), dirty=False, action=north
Step 21: pos=(0,3), dirty=False, action=east
Step 22: po

## Task 2:  Implement a simple reflex agent [10 Points]

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ Agents cannot directly use variable in the environment. They only gets the percepts as the arguments to the agent function. Use the function signature for the `simple_randomized_agent` function above.

In [ ]:
# Your code and description goes here
import random

def simple_reflex_agent(percept):
    """
    Một chương trình tác nhân phản xạ đơn giản.

    Tham số:
    percept (dict): Một dictionary chứa tri giác của tác nhân từ môi trường, bao gồm
                    'is_dirty' (bool), 'can_move' (dict với các hướng di chuyển), và
                    'location' (tuple).

    Trả về:
    str: Một hành động ('Suck', 'Move', hoặc 'NoOp').
    """
    # Bước 1: Kiểm tra xem có bụi bẩn ở vị trí hiện tại không.
    if percept['is_dirty']:
        return 'Suck'

    # Bước 2: Kiểm tra các hướng di chuyển có thể có.
    # Tác nhân sẽ tránh di chuyển vào tường (bumper sensor).
    # 'can_move' là một dictionary với các hướng (key) và giá trị bool (value).

    # Lấy ra danh sách các hướng di chuyển hợp lệ.
    valid_moves = [direction for direction, can_move in percept['can_move'].items() if can_move]

    # Bước 3: Đưa ra quyết định dựa trên các quy tắc phản xạ.
    if valid_moves:
        # Nếu có các hướng di chuyển hợp lệ, chọn ngẫu nhiên một hướng.
        return random.choice(valid_moves)
    else:
        # Nếu không có hướng di chuyển hợp lệ nào, không làm gì cả.
        return 'NoOp'


Show how the agent works with your environment.

In [ ]:
# Your code and description goes here
import numpy as np
import random
import time

# Tác nhân đã triển khai ở yêu cầu trước.
def simple_reflex_agent(percept):
    """
    Một chương trình tác nhân phản xạ đơn giản.
    """
    if percept['is_dirty']:
        return 'Suck'

    valid_moves = [direction for direction, can_move in percept['can_move'].items() if can_move]

    if valid_moves:
        return random.choice(valid_moves)
    else:
        return 'NoOp'

# Môi trường mô phỏng được triển khai để kiểm tra tác nhân
def simple_environment(agent_program, room_size=(5, 5), initial_dirt_ratio=0.5, max_steps=100, verbose=True):
    """
    Mô phỏng một môi trường phòng đơn giản.

    Tham số:
    agent_program (function): Hàm tác nhân được sử dụng để điều khiển robot.
    room_size (tuple): Kích thước của phòng (hàng, cột).
    initial_dirt_ratio (float): Tỷ lệ phần trăm các ô ban đầu bị bẩn.
    max_steps (int): Số bước tối đa trước khi dừng mô phỏng.
    verbose (bool): Nếu True, sẽ in ra trạng thái chi tiết của quá trình mô phỏng.

    Trả về:
    int: Số bước mà tác nhân đã thực hiện để hoàn thành nhiệm vụ.
    """
    rows, cols = room_size

    # 1. Khởi tạo môi trường
    # Tạo phòng sạch hoàn toàn (0 = sạch, 1 = bẩn)
    room = np.zeros(room_size, dtype=int)

    # Làm bẩn ngẫu nhiên một số ô
    num_dirty_squares = int(rows * cols * initial_dirt_ratio)
    dirty_indices = np.random.choice(range(rows * cols), num_dirty_squares, replace=False)
    for index in dirty_indices:
        room.flat[index] = 1

    # Vị trí ban đầu của robot
    robot_pos = (0, 0)

    if verbose:
        print("Môi trường được khởi tạo:")
        display_room(room, robot_pos)
        print("-" * 30)

    # Vòng lặp chính của môi trường
    action_count = 0
    while np.any(room == 1) and action_count < max_steps:
        # 2. Cung cấp tri giác cho tác nhân
        is_dirty = room[robot_pos] == 1

        can_move = {
            'up': robot_pos[0] > 0,
            'down': robot_pos[0] < rows - 1,
            'left': robot_pos[1] > 0,
            'right': robot_pos[1] < cols - 1
        }

        percept = {
            'is_dirty': is_dirty,
            'can_move': can_move,
            'location': robot_pos
        }

        # 3. Gọi chương trình tác nhân
        action = agent_program(percept)

        # 4. Phản ứng với hành động của tác nhân
        new_pos = robot_pos
        if action == 'Suck':
            room[robot_pos] = 0
            if verbose:
                print(f"Bước {action_count + 1}: Tại {robot_pos}, robot hút bụi.")
        elif action == 'up' and can_move['up']:
            new_pos = (robot_pos[0] - 1, robot_pos[1])
            if verbose:
                print(f"Bước {action_count + 1}: Robot di chuyển lên từ {robot_pos} tới {new_pos}.")
        elif action == 'down' and can_move['down']:
            new_pos = (robot_pos[0] + 1, robot_pos[1])
            if verbose:
                print(f"Bước {action_count + 1}: Robot di chuyển xuống từ {robot_pos} tới {new_pos}.")
        elif action == 'left' and can_move['left']:
            new_pos = (robot_pos[0], robot_pos[1] - 1)
            if verbose:
                print(f"Bước {action_count + 1}: Robot di chuyển trái từ {robot_pos} tới {new_pos}.")
        elif action == 'right' and can_move['right']:
            new_pos = (robot_pos[0], robot_pos[1] + 1)
            if verbose:
                print(f"Bước {action_count + 1}: Robot di chuyển phải từ {robot_pos} tới {new_pos}.")
        else:
            if verbose:
                print(f"Bước {action_count + 1}: Robot không làm gì (hành động: {action}).")

        robot_pos = new_pos
        action_count += 1

        if verbose:
            display_room(room, robot_pos)
            time.sleep(0.5) # Dừng một chút để dễ theo dõi
            print("-" * 30)

    # 5. Ghi nhận hiệu suất
    if not np.any(room == 1):
        if verbose:
            print(f"Nhiệm vụ hoàn thành! Robot đã mất {action_count} bước để làm sạch phòng.")
        return action_count
    else:
        if verbose:
            print(f"Đã đạt đến số bước tối đa ({max_steps}). Nhiệm vụ chưa hoàn thành.")
        return max_steps

def display_room(room, robot_pos):
    """Hàm trợ giúp để hiển thị trạng thái phòng."""
    display_grid = np.array(room, dtype='<U2')
    display_grid[robot_pos] = 'R'
    print(display_grid)

# Chạy mô phỏng
final_steps = simple_environment(simple_reflex_agent, verbose=True)

Môi trường được khởi tạo:
[['R' '1' '1' '0' '0']
 ['0' '1' '0' '1' '1']
 ['1' '0' '1' '0' '1']
 ['0' '0' '1' '0' '0']
 ['0' '0' '1' '0' '1']]
------------------------------
Bước 1: Tại (0, 0), robot hút bụi.
[['R' '1' '1' '0' '0']
 ['0' '1' '0' '1' '1']
 ['1' '0' '1' '0' '1']
 ['0' '0' '1' '0' '0']
 ['0' '0' '1' '0' '1']]
------------------------------
Bước 2: Robot di chuyển xuống từ (0, 0) tới (1, 0).
[['0' '1' '1' '0' '0']
 ['R' '1' '0' '1' '1']
 ['1' '0' '1' '0' '1']
 ['0' '0' '1' '0' '0']
 ['0' '0' '1' '0' '1']]
------------------------------
Bước 3: Robot di chuyển phải từ (1, 0) tới (1, 1).
[['0' '1' '1' '0' '0']
 ['0' 'R' '0' '1' '1']
 ['1' '0' '1' '0' '1']
 ['0' '0' '1' '0' '0']
 ['0' '0' '1' '0' '1']]
------------------------------
Bước 4: Tại (1, 1), robot hút bụi.
[['0' '1' '1' '0' '0']
 ['0' 'R' '0' '1' '1']
 ['1' '0' '1' '0' '1']
 ['0' '0' '1' '0' '0']
 ['0' '0' '1' '0' '1']]
------------------------------
Bước 5: Robot di chuyển xuống từ (1, 1) tới (2, 1).
[['0' '1' '1' 

### Cách hoạt động của tác nhân và môi trường

Triển khai trên minh họa một vòng lặp tác nhân-môi trường. Đây là cốt lõi của việc mô phỏng một hệ thống tác nhân:

Khởi tạo: Môi trường (simple_environment) thiết lập trạng thái ban đầu của căn phòng (sạch/bẩn) và vị trí của robot.

Vòng lặp: Mô phỏng chạy trong một vòng lặp while cho đến khi tất cả bụi bẩn được dọn sạch hoặc đạt đến giới hạn số bước.

Cảm biến (Percept): Ở mỗi bước, môi trường thu thập các thông tin cần thiết (ô hiện tại có bẩn không, có thể di chuyển không) và đóng gói chúng thành một tri giác (percept).

Hành động (Action): Tri giác này được truyền làm đầu vào cho hàm tác nhân (simple_reflex_agent). Tác nhân, chỉ dựa vào tri giác đó, sẽ đưa ra một hành động ('Suck', 'Move', 'NoOp').

Phản ứng (Reaction): Môi trường nhận hành động đó và cập nhật trạng thái của nó (làm sạch ô, di chuyển robot).

Quá trình này lặp đi lặp lại. Việc sử dụng hàm display_room và verbose giúp bạn dễ dàng theo dõi cách tác nhân đưa ra quyết định và phản ứng của môi trường sau mỗi bước.

## Task 3: Implement a model-based reflex agent [20 Points]

Model-based agents use a state to keep track of what they have done and perceived so far. Your agent needs to find out where it is located and then keep track of its current location. You also need a set of rules based on the state and the percepts to make sure that the agent will clean the whole room. For example, the agent can move to a corner to determine its location and then it can navigate through the whole room and clean dirty squares.

Describe how you define the __agent state__ and how your agent works before implementing it. ([Help with implementing state information on Python](https://github.com/mhahsler/CS7320-AI/blob/master/HOWTOs/store_agent_state_information.ipynb))

- Tác nhân phản xạ dựa trên mô hình (model-based reflex agent) này được thiết kế để làm sạch căn phòng một cách có hệ thống thay vì di chuyển ngẫu nhiên. Để làm được điều này, nó duy trì một trạng thái nội bộ để theo dõi thông tin về thế giới.
** Trạng thái Tác nhân (Agent State)
- Trạng thái của tác nhân được lưu trữ trong một dictionary Python và bao gồm các thông tin sau:
     + 'localized': Một biến boolean (True/False). Ban đầu là False. Nó sẽ trở thành True khi tác nhân đã xác định được vị trí của mình trong phòng.
     + 'location': Một tuple (x, y) biểu diễn tọa độ hiện tại của tác nhân. Ban đầu là None. Góc Tây-Bắc (trên-trái) được định nghĩa là (0, 0).
     + 'direction': Một chuỗi ('east' hoặc 'west') chỉ định hướng di chuyển hiện tại trong quá trình làm sạch theo hàng.
     + 'room_size': Một số nguyên n đại diện cho kích thước của căn phòng n x n. Tác nhân biết thông tin này trước.
** Logic hoạt động
- Tác nhân hoạt động theo hai giai đoạn riêng biệt:
1. Giai đoạn Định vị (Localization Phase):
     + Nếu trạng thái 'localized' là False, mục tiêu duy nhất của tác nhân là đi đến một góc tham chiếu đã biết.
     + Nó sẽ liên tục di chuyển về hướng west cho đến khi cảm biến va chạm bumpers['west'] báo hiệu đã chạm tường.
     + Sau đó, nó sẽ liên tục di chuyển về hướng north cho đến khi cảm biến va chạm bumpers['north'] báo hiệu đã chạm tường.
     + Khi cả hai điều kiện trên được thỏa mãn, tác nhân biết rằng nó đang ở góc Tây-Bắc (0, 0).
     + Nó cập nhật trạng thái: 'localized' thành True, 'location' thành (0, 0), và đặt hướng di chuyển ban đầu 'direction' là 'east'.
2. Giai đoạn Dọn dẹp (Cleaning Phase):
     + Một khi đã được định vị, tác nhân sẽ bắt đầu quá trình dọn dẹp có hệ thống.
     + Ưu tiên hàng đầu: Nếu cảm biến báo dirty (bẩn), hành động sẽ luôn là 'suck'. Tác nhân sẽ không di chuyển cho đến khi ô hiện tại đã sạch.
     + Di chuyển theo Mẫu (Pattern Movement): Nếu ô hiện tại đã sạch, tác nhân sẽ di chuyển theo một mẫu "cày ruộng" (boustrophedon):
          + Nếu hướng hiện tại là 'east', nó sẽ di chuyển sang phải cho đến khi chạm tường phía đông (x == n-1).
          + Khi chạm tường phía đông, nó sẽ di chuyển xuống một ô ('south') và đổi hướng thành 'west'.
          + Nếu hướng hiện tại là 'west', nó sẽ di chuyển sang trái cho đến khi chạm tường phía tây (x == 0).
          + Khi chạm tường phía tây, nó sẽ di chuyển xuống một ô ('south') và đổi hướng thành 'east'.
     + Quá trình này đảm bảo tác nhân sẽ đi qua mọi ô trong phòng một cách hiệu quả. Sau mỗi hành động di chuyển, tác nhân sẽ cập nhật tọa độ 'location' trong trạng thái nội bộ của mình.

In [ ]:
import numpy as np

agent_state = {}

def reset_model_based_agent_state(n=5):
    global agent_state
    agent_state = {
        'localized': False,
        'location': None,
        'direction': 'east',
        'room_size': n
    }

def model_based_reflex_agent(bumpers, dirty):
    global agent_state

    if not agent_state['localized']:
        if not bumpers['west']:
            return 'west'
        if not bumpers['north']:
            return 'north'

        agent_state['localized'] = True
        agent_state['location'] = (0, 0)

        return 'suck'

    if dirty:
        return 'suck'

    x, y = agent_state['location']
    n = agent_state['room_size']
    direction = agent_state['direction']
    action = None

    if direction == 'east':
        if x < n - 1:
            action = 'east'
            agent_state['location'] = (x + 1, y)
        else:
            action = 'south'
            agent_state['location'] = (x, y + 1)
            agent_state['direction'] = 'west'
    elif direction == 'west':
        if x > 0:
            action = 'west'
            agent_state['location'] = (x - 1, y)
        else:
            action = 'south'
            agent_state['location'] = (x, y + 1)
            agent_state['direction'] = 'east'

    current_y = agent_state['location'][1]
    if current_y >= n:
        return 'suck'

    return action


Show how the agent works with your environment.

In [ ]:
def environment(agent_function, n=5, p=0.2, max_steps=200, verbose=True):
    room = np.random.choice([0, 1], size=(n, n), p=[1-p, p])
    agent_pos = [np.random.randint(0, n), np.random.randint(0, n)]

    performance = 0

    def display_room(room_state, pos):
        grid = room_state.astype(str)
        grid[grid == '0'] = '.'
        grid[grid == '1'] = 'D'
        if 0 <= pos[0] < n and 0 <= pos[1] < n:
            grid[pos[0], pos[1]] = 'R'
        print(grid)
        print("-" * n * 2)

    for i in range(max_steps):
        if verbose:
            print(f"Step {i+1}")
            display_room(room, agent_pos)

        if np.sum(room) == 0:
            if verbose:
                print(f"\nAll squares are clean! Task finished.")
            return performance

        performance += 1

        row, col = agent_pos

        bumpers = {
            "north": row == 0,
            "south": row == n - 1,
            "west": col == 0,
            "east": col == n - 1
        }

        dirty = (room[row, col] == 1)

        action = agent_function(bumpers, dirty)

        if verbose:
            print(f"Agent position (row, col): {tuple(agent_pos)}, Dirty: {dirty}, Bumpers: {bumpers}")
            print(f"Action: {action}")

        if action == "suck":
            room[row, col] = 0
        elif action == "north" and not bumpers["north"]:
            agent_pos[0] -= 1
        elif action == "south" and not bumpers["south"]:
            agent_pos[0] += 1
        elif action == "west" and not bumpers["west"]:
            agent_pos[1] -= 1
        elif action == "east" and not bumpers["east"]:
            agent_pos[1] += 1

    if verbose:
        print("\nMax steps reached. Task failed.")
    return performance

if __name__ == "__main__":
    print("--- Running Model-Based Reflex Agent ---")
    reset_model_based_agent_state(n=5)
    np.random.seed(42)
    score = environment(model_based_reflex_agent, n=5, p=0.3, max_steps=100, verbose=True)
    print(f"\nTotal actions taken (Performance): {score}")

--- Running Model-Based Reflex Agent ---
Step 1
[['.' 'D' 'D' '.' '.']
 ['.' '.' 'D' '.' 'D']
 ['.' 'D' 'D' 'R' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']]
----------
Agent position (row, col): (2, 3), Dirty: False, Bumpers: {'north': False, 'south': False, 'west': False, 'east': False}
Action: west
Step 2
[['.' 'D' 'D' '.' '.']
 ['.' '.' 'D' '.' 'D']
 ['.' 'D' 'R' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']]
----------
Agent position (row, col): (2, 2), Dirty: True, Bumpers: {'north': False, 'south': False, 'west': False, 'east': False}
Action: west
Step 3
[['.' 'D' 'D' '.' '.']
 ['.' '.' 'D' '.' 'D']
 ['.' 'R' 'D' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']]
----------
Agent position (row, col): (2, 1), Dirty: True, Bumpers: {'north': False, 'south': False, 'west': False, 'east': False}
Action: west
Step 4
[['.' 'D' 'D' '.' '.']
 ['.' '.' 'D' '.' 'D']
 ['R' 'D' 'D' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']]
----------
Agent position (row, co

## Task 4: Simulation study [30 Points]

Compare the performance (the performance measure is defined in the PEAS description above) of the agents using  environments of different size. Do at least $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use 100 random runs for each. Present the results using tables and graphs. Discuss the differences between the agents.
([Help with charts and tables in Python](https://github.com/mhahsler/CS7320-AI/blob/master/HOWTOs/charts_and_tables.ipynb))

In [ ]:
import numpy as np
from collections import deque

# ===================== ENVIRONMENT =====================

def run_env(agent, n, p=0.2, steps=20000):
    room = np.random.rand(n, n) < p
    pos = [np.random.randint(n), np.random.randint(n)]
    act = 0
    while room.any() and act < steps:
        b = {"north": pos[0]==0, "south": pos[0]==n-1,
             "west": pos[1]==0, "east": pos[1]==n-1}
        d = room[pos[0], pos[1]]
        a = agent(b, d, pos)
        if a=="suck" and d:
            room[pos[0], pos[1]]=False
        elif a=="north" and not b["north"]:
            pos[0]-=1
        elif a=="south" and not b["south"]:
            pos[0]+=1
        elif a=="west" and not b["west"]:
            pos[1]-=1
        elif a=="east" and not b["east"]:
            pos[1]+=1
        act+=1
    return act


# ===================== AGENTS =====================

acts=["north","south","west","east","suck"]

def ag_ran(b,d,pos):
    return np.random.choice(acts)

def ag_ref(b,d,pos):
    if d: return "suck"
    dirs=[k for k,v in b.items() if not v]
    return np.random.choice(dirs) if dirs else "suck"


# ===================== MODEL-BASED REFLEX AGENT =====================

class AgModel:
    def __init__(self,n):
        self.n=n
        self.known=np.full((n,n),None)  # None=unknown, False=clean, True=dirty
        self.path=[]

    def bfs(self,start,targets):
        n=self.n
        q=deque([(start,[])])
        seen={tuple(start)}
        dirs={"north":(-1,0),"south":(1,0),"west":(0,-1),"east":(0,1)}
        while q:
            (x,y),path=q.popleft()
            if (x,y) in targets:
                return path
            for a,(dx,dy) in dirs.items():
                nx,ny=x+dx,y+dy
                if 0<=nx<n and 0<=ny<n and (nx,ny) not in seen:
                    seen.add((nx,ny))
                    q.append(((nx,ny),path+[a]))
        return []

    def __call__(self,b,d,pos):
        x,y=pos
        self.known[x,y]=d
        if d:
            self.path=[]
            return "suck"
        n=self.n
        unknowns={(i,j) for i in range(n) for j in range(n) if self.known[i,j] is None}
        dirties={(i,j) for i in range(n) for j in range(n) if self.known[i,j]}
        if not self.path:
            targets=dirties if dirties else unknowns
            if targets:
                self.path=self.bfs((x,y),targets)
        if not self.path:
            return "suck"
        move=self.path.pop(0)
        return move


# ===================== TEST =====================

def test(agent,n,rep=30):
    res=[]
    for _ in range(rep):
        a=agent if agent!=AgModel else AgModel(n)
        res.append(run_env(a,n))
    return np.mean(res)

sizes=[5,10,100]
print("Running simulations...")
print("Size\tRandom\tReflex\tModel-based")
for n in sizes:
    res_ran=test(ag_ran,n)
    res_ref=test(ag_ref,n)
    res_mod=test(AgModel,n)
    print(f"{n}x{n}\t{res_ran:.1f}\t{res_ref:.1f}\t{res_mod:.1f}")

Running simulations...
Size	Random	Reflex	Model-based
5x5	434.2	90.5	27.2
10x10	2837.0	1036.5	119.9
100x100	20000.0	20000.0	12059.1


Fill out the following table with the average performance measure for 100 random runs (you may also create this table with code):

| Size     | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|----------|------------------|---------------------|--------------------------|
| 5x5     | | | |
| 10x10   | | | |
| 100x100 | | | |

Add charts to compare the performance of the different agents.

In [ ]:
# Your graphs and discussion of the results goes here
plt.bar(x-w,res_ran,w,label="Ran")
plt.bar(x,res_ref,w,label="Ref")
plt.bar(x+w,res_mod,w,label="Mod")
plt.xticks(x,sizes)
plt.ylabel("Avg Actions")
plt.xlabel("Room Size")
plt.legend()
plt.show()
import textwrap

txt = "Kết quả cho thấy Simple Reflex agent hoạt động tốt nhất ở phòng nhỏ và trung bình, trong khi Randomized chỉ hiệu quả ở phòng nhỏ, còn Model-based Reflex lại tốn nhiều bước hơn dự kiến. Vậy tại sao Model-based Reflex không vượt trội? Nguyên nhân là do cách xây dựng mô hình chưa tối ưu, khiến agent thực hiện nhiều hành động dư thừa. Ở phòng lớn 100×100, cả ba agent đều đạt giới hạn bước, đặt ra câu hỏi: liệu các chiến lược phản xạ có còn phù hợp với môi trường quá rộng? Câu trả lời là không, và để cải thiện cần hướng đến các phương pháp có khả năng tìm đường và lập kế hoạch tốt hơn. Thực tế, trong robot hút bụi, người ta thường kết hợp cảm biến với bản đồ (mapping) và thuật toán tìm đường để dọn sạch nhanh và hiệu quả hơn thay vì chỉ dựa vào phản xạ."

print(textwrap.fill(txt, width=100))

NameError: name 'plt' is not defined

## Task 5: Robustness of the agent implementations [10 Points]

Describe how **your agent implementations** will perform

* if it is put into a rectangular room with unknown size,
* if the cleaning area can have an irregular shape (e.g., a hallway connecting two rooms), or
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).
* if the dirt sensor is not perfect and gives 10% of the time a wrong reading (clean when it is dirty or dirty when it is clean).
* if the bumper sensor is not perfect and 10% of the time does not report a wall when there is one.

1. Nếu phòng là hình chữ nhật nhưng không biết kích thước
  - Tác nhân Ngẫu nhiên (Randomized Agent): Tác nhân này không quan tâm đến kích thước hay hình dạng của phòng. Nó sẽ tiếp tục di chuyển ngẫu nhiên và va vào tường như bình thường. Hiệu suất của nó vốn đã rất tệ, nên sẽ không có gì thay đổi lớn.
  - Tác nhân Phản xạ Đơn giản (Simple Reflex Agent): Tác nhân này hoạt động khá tốt. Nó không cần biết kích thước phòng vì nó chỉ phản ứng với các bức tường ngay trước mặt. Nó sẽ đi lang thang trong phòng hình chữ nhật và dọn dẹp một cách ngẫu nhiên. Nó vẫn không hiệu quả, nhưng nó sẽ không bị "hỏng".
  - Tác nhân Dựa trên Mô hình (Model-based Reflex Agent): Sẽ thất bại hoàn toàn. Tác nhân này được lập trình với giả định về một căn phòng hình vuông `n x n`. Nếu kích thước thực tế khác đi (ví dụ 5x8 thay vì 5x5), kế hoạch di chuyển theo hàng của nó sẽ sai. Nó sẽ nghĩ rằng nó đã đến cuối hàng trong khi thực tế chưa, làm cho tọa độ nội bộ của nó bị lệch hoàn toàn so với vị trí thực. Kế hoạch dọn dẹp có hệ thống sẽ sụp đổ.
2. Nếu khu vực dọn dẹp có hình dạng bất thường (ví dụ: có hành lang)
  - Tác nhân Ngẫu nhiên: Giống như trên, nó không quan tâm. Nó sẽ đi lang thang một cách ngẫu nhiên trong mọi không gian mà nó có thể vào được.
  - Tác nhân Phản xạ Đơn giản: Tác nhân này xử lý tốt các hình dạng bất thường. Đối với nó, một hành lang chỉ là một không gian hẹp với các bức tường. Nó sẽ đi lang thang qua hành lang và vào các phòng khác một cách tự nhiên. Đây là ưu điểm lớn của việc không có một kế hoạch cứng nhắc.
  - Tác nhân Dựa trên Mô hình: Sẽ thất bại hoàn toàn. Kế hoạch di chuyển "cày ruộng" của nó chỉ hoạt động trên một hình chữ nhật trống. Khi gặp một bức tường bất ngờ ở giữa (như ở góc của một phòng hình chữ L) hoặc một hành lang hẹp, logic di chuyển của nó sẽ bị phá vỡ. Nó không có khả năng tự tìm đường trong các không gian phức tạp.
3. Nếu trong phòng có chướng ngại vật (đồ đạc)
  - Tác nhân Ngẫu nhiên: Nó sẽ liên tục cố gắng di chuyển vào chướng ngại vật vì nó phớt lờ cảm biến va chạm. Điều này làm cho nó càng kém hiệu quả hơn.
  - Tác nhân Phản xạ Đơn giản: Tác nhân này xem chướng ngại vật như những bức tường nhỏ. Nó sẽ va vào chúng, cảm nhận được, và thử một hướng đi khác. Nó có thể di chuyển xung quanh chướng ngại vật, nhưng cũng có thể bị kẹt trong một không gian hẹp giữa chướng ngại vật và tường.
  - Tác nhân Dựa trên Mô hình: Sẽ thất bại hoàn toàn. Kế hoạch của nó yêu cầu các hàng di chuyển không bị cản trở. Một chướng ngại vật ở giữa đường đi sẽ làm hỏng toàn bộ lộ trình. Tác nhân không có logic để đi vòng qua một vật cản bất ngờ; nó sẽ coi đó là bức tường cuối phòng và làm sai lệch toàn bộ bản đồ nội bộ của nó.
4. Nếu cảm biến bụi bẩn không hoàn hảo (sai 10%)
  - Tác nhân Ngẫu nhiên: Không bị ảnh hưởng, vì nó vốn dĩ không sử dụng cảm biến này để ra quyết định.
  - Tác nhân Phản xạ Đơn giản: Đây là một vấn đề lớn. Nếu cảm biến báo "sạch" ở một nơi "bẩn" (false negative), nó sẽ bỏ qua vết bẩn đó. Căn phòng có thể sẽ không bao giờ được dọn sạch hoàn toàn. Nếu cảm biến báo "bẩn" ở một nơi "sạch" (false positive), nó sẽ lãng phí một lượt để hút bụi một ô đã sạch. Điều này làm giảm hiệu suất.
  - Tác nhân Dựa trên Mô hình: Tương tự như tác nhân phản xạ đơn giản. Lộ trình di chuyển của nó vẫn hoàn hảo, nhưng việc dọn dẹp sẽ không đáng tin cậy. Nó sẽ bỏ sót các vết bẩn (false negative) hoặc lãng phí năng lượng (false positive). Mục tiêu làm sạch hoàn toàn căn phòng sẽ không đạt được.
5. Nếu cảm biến va chạm không hoàn hảo (không báo có tường 10%)
  - Tác nhân Ngẫu nhiên: Không bị ảnh hưởng, vì nó phớt lờ cảm biến này.
  - Tác nhân Phản xạ Đơn giản: Nó sẽ cố gắng di chuyển xuyên tường. Môi trường sẽ ngăn nó lại, nhưng nó đã lãng phí một hành động. Điều này làm giảm hiệu suất nhưng không làm hỏng hoàn toàn tác nhân.
  - Tác nhân Dựa trên Mô hình: THẢM HỌA. Đây là trường hợp tệ nhất. Trong giai đoạn định vị, nếu nó không "nhìn thấy" bức tường phía tây hoặc phía bắc, nó sẽ không bao giờ biết mình đang ở góc và sẽ bị kẹt trong một vòng lặp vô tận. Trong giai đoạn dọn dẹp, nếu nó đến cuối một hàng và cảm biến không hoạt động, tọa độ nội bộ của nó sẽ bị cập nhật sai. Từ đó, toàn bộ bản đồ trong đầu của nó sẽ bị lệch so với thực tế và tác nhân sẽ hoàn toàn bị "lạc".

## Advanced task: Imperfect Dirt Sensor

* __Graduate students__ need to complete this task [10 points]
* __Undergraduate students__ can attempt this as a bonus task [max +5 bonus points].

1. Change your simulation environment to run experiments for the following problem: The dirt sensor has a 10% chance of giving the wrong reading. Perform experiments to observe how this changes the performance of the three implementations. Your model-based reflex agent is likely not able to clean the whole room, so you need to measure performance differently as a tradeoff between energy cost and number of uncleaned squares.

2. Design an implement a solution for your model-based agent that will clean better. Show the improvement with experiments.

In [ ]:
# Your code and discussion goes here
import random

class MôiTrường:
    def __init__(self, kích_thước=(10, 10), độ_bẩn_ban_đầu=0.5):
        self.kích_thước = kích_thước
        self.grid = [[0] * kích_thước[1] for _ in range(kích_thước[0])]
        self.vị_trí_tác_nhân = (0, 0)
        self.năng_lượng = 0
        self.tạo_bẩn(độ_bẩn_ban_đầu)

    def tạo_bẩn(self, độ_bẩn):
        for i in range(self.kích_thước[0]):
            for j in range(self.kích_thước[1]):
                if random.random() < độ_bẩn:
                    self.grid[i][j] = 1 # 1 = bẩn, 0 = sạch

    def cảm_nhận_bụi_bẩn(self, tỉ_lệ_sai=0.1):
        x, y = self.vị_trí_tác_nhân
        trạng_thái_thực = self.grid[x][y]
        if random.random() < tỉ_lệ_sai:
            return 1 - trạng_thái_thực # Trả về trạng thái sai
        return trạng_thái_thực # Trả về trạng thái đúng

    def di_chuyển_và_hút(self, hướng):
        x, y = self.vị_trí_tác_nhân
        năng_lượng_tiêu_thụ = 1

        if hướng == "hút":
            if self.grid[x][y] == 1:
                self.grid[x][y] = 0
                năng_lượng_tiêu_thụ += 10 # Chi phí hút bụi cao hơn
        else: # Di chuyển
            dx, dy = 0, 0
            if hướng == "lên": dx = -1
            elif hướng == "xuống": dx = 1
            elif hướng == "trái": dy = -1
            elif hướng == "phải": dy = 1

            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < self.kích_thước[0] and 0 <= new_y < self.kích_thước[1]:
                self.vị_trí_tác_nhân = (new_x, new_y)
            else:
                năng_lượng_tiêu_thụ += 5 # Chi phí va chạm tường

        self.năng_lượng += năng_lượng_tiêu_thụ
        return năng_lượng_tiêu_thụ

    def đếm_ô_chưa_sạch(self):
        chưa_sạch = sum(sum(row) for row in self.grid)
        return chưa_sạch

class TácNhân:
    def __init__(self, môi_trường):
        self.môi_trường = môi_trường

    def hành_động(self, trạng_thái):
        pass

class TácNhânPhảnXạ(TácNhân):
    def hành_động(self, trạng_thái):
        if trạng_thái == 1:
            return "hút"
        return random.choice(["lên", "xuống", "trái", "phải"])

class TácNhânDựaTrênMôHình(TácNhân):
    def __init__(self, môi_trường):
        super().__init__(môi_trường)
        self.bản_đồ_nội_tâm = [[-1] * self.môi_trường.kích_thước[1] for _ in range(self.môi_trường.kích_thước[0])] # -1 = chưa khám phá, 0 = sạch, 1 = bẩn
        self.vị_trí_đã_thăm = set()
        self.lịch_sử_di_chuyển = []

    def hành_động(self, trạng_thái):
        x, y = self.môi_trường.vị_trí_tác_nhân
        self.bản_đồ_nội_tâm[x][y] = trạng_thái
        self.vị_trí_đã_thăm.add((x, y))

        if trạng_thái == 1:
            return "hút"

        # Di chuyển tới ô chưa được khám phá
        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.môi_trường.kích_thước[0] and 0 <= ny < self.môi_trường.kích_thước[1] and self.bản_đồ_nội_tâm[nx][ny] == -1:
                self.lịch_sử_di_chuyển.append((x, y))
                if dx == 1: return "xuống"
                if dx == -1: return "lên"
                if dy == 1: return "phải"
                if dy == -1: return "trái"

        # Nếu đã khám phá hết, quay lại các ô bẩn (nếu có)
        for i in range(self.môi_trường.kích_thước[0]):
            for j in range(self.môi_trường.kích_thước[1]):
                if self.bản_đồ_nội_tâm[i][j] == 1:
                    # Logic di chuyển tới (i, j)
                    pass

        # Hoặc quay lại lịch sử di chuyển
        if self.lịch_sử_di_chuyển:
            self.lịch_sử_di_chuyển.pop()
            # Logic di chuyển ngược lại

        return random.choice(["lên", "xuống", "trái", "phải"])

class TácNhânDựaTrênMôHìnhCảiTiến(TácNhânDựaTrênMôHình):
    def hành_động(self, trạng_thái):
        x, y = self.môi_trường.vị_trí_tác_nhân
        self.bản_đồ_nội_tâm[x][y] = trạng_thái
        self.vị_trí_đã_thăm.add((x, y))

        # Cải tiến: Nếu cảm biến báo bẩn, hút ngay lập tức và ghi nhớ vị trí này
        # Tránh việc di chuyển sang ô khác mà không hút, sau đó quay lại và có thể bỏ sót
        if trạng_thái == 1:
            return "hút"

        # Tìm ô chưa được khám phá hoặc có thể là ô bẩn (dựa trên bản đồ nội tâm)
        # Sử dụng thuật toán tìm đường đi đơn giản (BFS/DFS) hoặc chỉ đơn giản là đi theo thứ tự

        # Chiến lược di chuyển theo một đường zig-zag để đảm bảo phủ hết diện tích
        if (x, y) == (self.môi_trường.kích_thước[0] - 1, self.môi_trường.kích_thước[1] - 1):
             return None # Kết thúc

        if y % 2 == 0: # Di chuyển sang phải
            if y < self.môi_trường.kích_thước[1] - 1:
                return "phải"
            else:
                return "xuống"
        else: # Di chuyển sang trái
            if y > 0:
                return "trái"
            else:
                return "xuống"

class TácNhânMụcTiêu(TácNhân):
    def __init__(self, môi_trường):
        super().__init__(môi_trường)
        self.mục_tiêu = []
        for i in range(self.môi_trường.kích_thước[0]):
            for j in range(self.môi_trường.kích_thước[1]):
                if self.môi_trường.grid[i][j] == 1:
                    self.mục_tiêu.append((i, j))

    def hành_động(self, trạng_thái):
        if self.mục_tiêu:
            x, y = self.môi_trường.vị_trí_tác_nhân
            mx, my = self.mục_tiêu[0]
            if (x, y) == (mx, my):
                self.mục_tiêu.pop(0)
                return "hút"

            # Di chuyển tới mục tiêu đầu tiên
            if x < mx: return "xuống"
            if x > mx: return "lên"
            if y < my: return "phải"
            if y > my: return "trái"

        return None # Đã hoàn thành



### Chạy các thử nghiệm


def chạy_thử_nghiệm(tên_tác_nhân, tác_nhân, môi_trường, số_bước_tối_đa=200):
    môi_trường_thử = MôiTrường(kích_thước=môi_trường.kích_thước, độ_bẩn_ban_đầu=0.5)
    tác_nhân_thử = tác_nhân(môi_trường_thử)

    for bước in range(số_bước_tối_đa):
        trạng_thái_cảm_nhận = môi_trường_thử.cảm_nhận_bụi_bẩn(tỉ_lệ_sai=0.1)
        hành_động = tác_nhân_thử.hành_động(trạng_thái_cảm_nhận)
        if hành_động is None:
            break
        môi_trường_thử.di_chuyển_và_hút(hành_động)

    chi_phí_năng_lượng = môi_trường_thử.năng_lượng
    ô_chưa_sạch = môi_trường_thử.đếm_ô_chưa_sạch()

    print(f"--- Kết quả cho {tên_tác_nhân} ---")
    print(f"Chi phí năng lượng: {chi_phí_năng_lượng}")
    print(f"Số ô chưa sạch: {ô_chưa_sạch}")
    print(f"Đánh đổi: {chi_phí_năng_lượng} + {ô_chưa_sạch*100} (giả định 1 ô chưa sạch = 100 năng lượng)")
    print("---------------------------------")
    return chi_phí_năng_lượng, ô_chưa_sạch

def main():
    môi_trường = MôiTrường()

    print("Chạy thử nghiệm với cảm biến có 10% khả năng đưa ra kết quả sai")

    chạy_thử_nghiệm("Tác nhân phản xạ", TácNhânPhảnXạ, môi_trường)
    chạy_thử_nghiệm("Tác nhân dựa trên mô hình (cơ bản)", TácNhânDựaTrênMôHình, môi_trường)
    chạy_thử_nghiệm("Tác nhân mục tiêu", TácNhânMụcTiêu, môi_trường)
    chạy_thử_nghiệm("Tác nhân dựa trên mô hình (cải tiến)", TácNhânDựaTrênMôHìnhCảiTiến, môi_trường)

if __name__ == "__main__":
    main()

## More Advanced Implementation (not for credit)

If the assignment was to easy for yuo then you can think about the following problems. These problems are challenging and not part of this assignment. We will learn implementation strategies and algorithms useful for these tasks during the rest of the semester.

* __Obstacles:__ Change your simulation environment to run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Perform experiments to observe how this changes the performance of the three implementations. Describe what would need to be done to perform better with obstacles. Add code if you can.

* __Agent for and environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square (note that this is actually depth-first search).

* __Utility-based agent:__ Change the environment for a $5 \times 5$ room, so each square has a fixed probability of getting dirty again. For the implementation, we give the environment a 2-dimensional array of probabilities. The utility of a state is defined as the number of currently clean squares in the room. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 100000 time steps. To do this, the agent needs to learn the probabilities with which different squares get dirty again. This is very tricky!

In [ ]:
# Mỗi lần chạy môi trường, thêm các ô ngẫu nhiên là obstacle.
# Obstacle sẽ kích hoạt cảm biến bumper (như tường).
# Agent không biết trước vị trí obstacle.
# Cần quan sát hiệu năng của các agent đã code trước đó (simple reflex, model-based reflex, random, utility...).
# Để cải thiện:
# Agent nên lưu lại bản đồ để tránh đi vào obstacle nhiều lần.
# Có thể áp dụng path planning (DFS/BFS/A*) khi muốn đến ô khác mà bị chặn.

import random

class EnvironmentWithObstacles:
    def __init__(self, width, height, n_obstacles):
        self.width = width
        self.height = height
        self.grid = [['clean' for _ in range(width)] for _ in range(height)]
        # Thêm chướng ngại vật
        self.obstacles = set()
        while len(self.obstacles) < n_obstacles:
            x, y = random.randrange(width), random.randrange(height)
            self.obstacles.add((x, y))

    def is_obstacle(self, pos):
        return pos in self.obstacles

    def status(self, pos):
        if self.is_obstacle(pos):
            return 'obstacle'
        return self.grid[pos[1]][pos[0]]

    def dirty_random(self, prob=0.05):
        for y in range(self.height):
            for x in range(self.width):
                if (x,y) not in self.obstacles and random.random() < prob:
                    self.grid[y][x] = 'dirty'


In [ ]:
# Môi trường không biết kích thước, không biết vị trí bắt đầu, và có obstacle.
# Một chiến lược đơn giản:
# Luôn di chuyển tới ô chưa thăm / chưa dọn gần nhất (giống DFS exploration).
# Lưu bản đồ đã phát hiện vào một dict/map.
# Tức là agent vừa khám phá, vừa dọn.

from collections import deque

class ExploringAgent:
    def __init__(self):
        self.known_map = {}    # (x,y) -> "clean"/"dirty"/"obstacle"
        self.visited = set()
        self.frontier = set()
        self.path = []         # danh sách các bước cần đi
        self.pos = (0,0)

    def neighbors(self, pos):
        x, y = pos
        return [(x+1,y), (x-1,y), (x,y+1), (x,y-1)]

    def update_frontier(self):
        for v in self.visited:
            for n in self.neighbors(v):
                if n not in self.visited and self.known_map.get(n) != "obstacle":
                    self.frontier.add(n)

    def bfs_path(self, start, goal):
      queue = deque([(start, [])])
      seen = {start}
      while queue:
        cur, path = queue.popleft()
        if cur == goal:
            return path
        for n in self.neighbors(cur):
            if n not in seen:
                # Nếu chưa biết gì về ô n, vẫn cho phép đi qua (unknown = free)
                if self.known_map.get(n) != "obstacle":
                    seen.add(n)
                    queue.append((n, path+[n]))
      return None


    def act(self, percept):
        pos, dirty, bumper = percept
        self.pos = pos

        # Cập nhật bản đồ và visited
        self.visited.add(pos)
        self.known_map[pos] = "dirty" if dirty else "clean"

        # Nếu ô này đang bẩn thì dọn trước
        if dirty:
            return "suck"

        # Nếu có sẵn đường đi thì tiếp tục bước tiếp theo
        if self.path:
            next_step = self.path.pop(0)
            dx, dy = next_step[0]-pos[0], next_step[1]-pos[1]
            if dx == 1: return "move_right"
            if dx == -1: return "move_left"
            if dy == 1: return "move_down"
            if dy == -1: return "move_up"

        # Cập nhật frontier
        self.update_frontier()

        # Nếu còn frontier → chọn một mục tiêu và lập kế hoạch BFS
        if self.frontier:
            target = min(self.frontier, key=lambda f: abs(f[0]-pos[0])+abs(f[1]-pos[1]))
            self.frontier.remove(target)
            path = self.bfs_path(pos, target)
            if path:
                self.path = path
                return self.act(percept)  # gọi lại để lấy action đầu tiên

        # Nếu không còn gì để làm
        return "wait"


In [ ]:
# Mỗi ô (i,j) có probability p(i,j) để bẩn lại sau mỗi bước.

# Utility của trạng thái = số ô sạch hiện tại.

# Agent cần học ước lượng xác suất bẩn lại (thay vì biết trước).

# Sau nhiều quan sát, agent có thể ước lượng xác suất bằng tần suất:

#p^ (i, j) = (số lần ô (i,j) bẩn sau khi được dọn) / (số lần dọn ô (i,j))

# Khi chọn hành động, agent chọn nước đi sao cho tối đa hóa kỳ vọng utility trong 100000 bước.

import numpy as np

class ProbabilisticEnvironment:
    def __init__(self, size=5):
        self.size = size
        # mỗi ô có xác suất riêng
        self.probs = np.random.rand(size, size)
        self.grid = np.zeros((size, size))  # 0 = clean, 1 = dirty

    def step(self):
        # mỗi bước: có thể bẩn lại
        for i in range(self.size):
            for j in range(self.size):
                if np.random.rand() < self.probs[i,j]:
                    self.grid[i,j] = 1

class UtilityAgent:
    def __init__(self, size=5):
        self.size = size
        self.estimate = np.zeros((size,size))
        self.clean_counts = np.zeros((size,size))
        self.re_dirty_counts = np.zeros((size,size))

    def update_estimate(self, pos, dirty_before, dirty_after):
        i,j = pos
        if dirty_before and not dirty_after:
            self.clean_counts[i,j] += 1
        if not dirty_before and dirty_after:
            self.re_dirty_counts[i,j] += 1
        # ước lượng
        self.estimate[i,j] = self.re_dirty_counts[i,j] / max(1,self.clean_counts[i,j])

    def act(self, pos, env):
        i,j = pos
        if env.grid[i,j] == 1:
            env.grid[i,j] = 0
            return 'suck'
        # chọn ô bẩn có xác suất cao nhất để di chuyển tới
        targets = np.argwhere(env.grid==1)
        if len(targets)==0:
            return 'wait'
        target = targets[np.random.randint(len(targets))]
        return f'move_to {tuple(target)}'


Code mô phỏng để test

In [ ]:
import random

class SimpleEnvironment:
    def __init__(self, width=7, height=7, n_obstacles=5, dirt_prob=0.2, seed=42):
        random.seed(seed)
        self.width = width
        self.height = height
        self.agent_pos = (0, 0)
        self.steps = 0

        # Khởi tạo lưới: "clean" hoặc "dirty"
        self.grid = {}
        for x in range(width):
            for y in range(height):
                self.grid[(x, y)] = "dirty" if random.random() < dirt_prob else "clean"

        # Thêm obstacles
        self.obstacles = set()
        while len(self.obstacles) < n_obstacles:
            ox, oy = random.randrange(width), random.randrange(height)
            if (ox, oy) != self.agent_pos:
                self.obstacles.add((ox, oy))
                self.grid[(ox, oy)] = "obstacle"

    def sense(self):
        """Trả về percept cho agent"""
        pos = self.agent_pos
        dirty = self.grid.get(pos) == "dirty"
        bumper = False  # cập nhật sau khi thử di chuyển
        return (pos, dirty, bumper)

    def step(self, action):
        """Thực hiện action"""
        x, y = self.agent_pos
        bumper = False

        if action == "suck":
            if self.grid.get((x, y)) == "dirty":
                self.grid[(x, y)] = "clean"
                return (self.agent_pos, False, False), 10  # reward dọn sạch
            return (self.agent_pos, False, False), -1

        new_pos = None
        if action == "move_up":
            new_pos = (x, y - 1)
        elif action == "move_down":
            new_pos = (x, y + 1)
        elif action == "move_left":
            new_pos = (x - 1, y)
        elif action == "move_right":
            new_pos = (x + 1, y)

        if new_pos:
            if (0 <= new_pos[0] < self.width and
                0 <= new_pos[1] < self.height and
                new_pos not in self.obstacles):
                self.agent_pos = new_pos
            else:
                bumper = True  # đụng obstacle hoặc tường

        dirty = self.grid.get(self.agent_pos) == "dirty"
        self.steps += 1
        return (self.agent_pos, dirty, bumper), -0.1  # chi phí di chuyển nhẹ

    def render(self):
        """In ra lưới"""
        for y in range(self.height):
            row = []
            for x in range(self.width):
                if (x, y) == self.agent_pos:
                    row.append("A")  # agent
                elif (x, y) in self.obstacles:
                    row.append("X")  # obstacle
                elif self.grid[(x, y)] == "dirty":
                    row.append("*")
                else:
                    row.append(".")
            print(" ".join(row))
        print()


In [ ]:
def run_simulation(agent, env, max_steps=50):
    for step in range(max_steps):
        percept = env.sense()
        action = agent.act(percept)
        percept, reward = env.step(action)
        env.render()
        print(f"Step {step}: Action={action}, Reward={reward}, Pos={env.agent_pos}")
        print("-" * 40)

# Test
env = SimpleEnvironment(width=7, height=7, n_obstacles=7, dirt_prob=0.3, seed=1)
agent = ExploringAgent()

run_simulation(agent, env, max_steps=30)
